## Kelley hu4d5-5 VL-T93A error analysis

In this notebook we will use a random forest model to find the most energetically influential degrees of freedom for the VL-T93A TI production run. Next, we will compare the sampling of these DOF during TI production to a free energy profile derived from end state GaMD sampling. We will attempt to  correct any inaccurate sampling in the TI data and find the estimated ddG before and after the correction. 

In [1]:
import os
os.chdir("..")
from common_functions import *

### Ingesting initial TI data

In [2]:
os.chdir("./TI_data/VL-T93A/")
geom_dvdls = pd.read_csv("T93A_bound.csv")
dvdls_ub = pd.read_csv("T93A_unbound.csv")

### Original ddG estimate:

In [3]:
orig_dG_bd = geom_dvdls.groupby("Lambda").mean().sum()["weight_dvdl"]
orig_dG_ubd = dvdls_ub.groupby("Lambda").mean().sum()["weight_dvdl"]

empirical_value = 0.83
orig_error = abs((orig_dG_bd - orig_dG_ubd) - empirical_value)

print("Original ddG estimate: ")
print(f"{round(orig_dG_bd - orig_dG_ubd, 4)} kcal/mol ")
print()
print("Original ddG error: ")
print(f"{round(orig_error, 4)} kcal/mol")

Original ddG estimate: 
1.0126 kcal/mol 

Original ddG error: 
0.1826 kcal/mol


### Removing correlated variables among candidate degrees of freedom

This is important because we want to limit the noise in our model training. See our methods/supplemental methods section for our process to choose the input features.

In [4]:
X = geom_dvdls.drop([
    "#Frame", "weight_dvdl", "dvdl", "Run", "Lambda", "Q445_chi3"
], axis=1)

Y = geom_dvdls["weight_dvdl"]

X_scl = pd.DataFrame(StandardScaler().fit_transform(X))
X_scl.columns = X.columns


### Checking to see if there is any cross-correlations within the independent variables

In [5]:
absCorr = abs(X_scl.corr())
for i in absCorr.columns:
    for j in absCorr.index:
        cor = absCorr.loc[i, j]
        if abs(cor) > 0.5 and i != j:
            print(i, j)
            print(cor)
            

### Using random forest model to identify the most energetically influential degrees of freedom

We run our model 25 times, then sort the results by the mean of feature importance across the 25 iterations. For this particular perturbation, the model $R^2$ between the geometric DOF (nearby side chain rotamers or interatomic distances) and the energetic DV/DL was not strong enough for us to check the sampling of these degrees of freedom. 

In [14]:
rfeDefault = RFE(estimator=DecisionTreeRegressor(max_depth=5, random_state=42), n_features_to_select=0.75, step=0.05)
rfDefault = RandomForestRegressor(
    max_depth=10, n_estimators=200, oob_score=True, max_features=0.6, min_samples_leaf = 7, min_samples_split=14, random_state=42
)

pipelineDefault_rf = Pipeline([
    ('feature_scaling', StandardScaler()),
    ('feature_selection', rfeDefault),
    ('regression_model', rfDefault)
])


imps = benchmark_model(pipelineDefault_rf, X_scl, Y, geom_dvdls["Lambda"])
imps[["Mean", "Median"]].sort_values(by="Mean", ascending=False)[:15]

Avg. training r2: 
0.4617
Training r2 std dev: 
0.0008
Avg. test r2: 
0.3396
Testing r2 std dev: 
0.0045


,Mean,Median
Y447_chi2,0.183844,0.286733
asp73_t228,0.164836,0.164479
T448_chi1,0.133154,0.030792
Q74_chi3,0.073975,0.078537
D73_chi1,0.072385,0.072463
Q74_chi2,0.057832,0.049129
I357_chi2,0.054338,0.056122
K82_chi1,0.046404,0.059724
K82_chi4,0.041107,0.054734
Y447_chi1,0.028395,0.026853
